<a href="https://colab.research.google.com/github/danfolador/recpadroes/blob/main/RecPad_Exercicio08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#=====================================================================
# INICIALIZACAO
#=====================================================================
import pandas as pd
import numpy as np

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

#train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')

test = pd.read_csv('/content/drive/MyDrive/!MestradoTemp/03 Reconhecimento Padroes/Exercicio08_AjusteCaract/test.csv')
train = pd.read_csv('/content/drive/MyDrive/!MestradoTemp/03 Reconhecimento Padroes/Exercicio08_AjusteCaract/train.csv')

X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]
y = np.ravel(y)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#=====================================================================
# PRONOMES
#=====================================================================
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        return Xdrop

In [4]:
#=====================================================================
# ATRIBUTOS NUMERICOS
#=====================================================================
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()

In [5]:
#=====================================================================
# EXTRAI CATEGORICOS
#=====================================================================
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [6]:
#=====================================================================
# TRATA ATRIBUTOS
#=====================================================================
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
            ('unecaracteristicas', FeatureUnion([
            ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
])),
            ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [10]:
#=====================================================================
# MODELO TOTAL
#=====================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np



pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True],
    'classificador__min_samples_split': [4,6,7],
    'classificador__max_depth': [5,7,8],
    'classificador__criterion': ['gini'],
    'classificador__max_features':['auto', 'log2'],
    'classificador__min_samples_leaf':[1]
}



modelo = GridSearchCV(pipetotal,
                      param_grid=parametros,
                      scoring='roc_auc',
                      n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())

modelo.fit(X,y)
score_med=np.mean(scores['test_score'])

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())

print('--------------------------------')
print('score med=',np.mean(scores['test_score']))

--------------------------------
score med= 0.8611965267809889


In [11]:
#=====================================================================
# AJUSTE
#=====================================================================
modelo.fit(X,y)

print('--------------------------------')
print(modelo.best_params_)

y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('results_daniel.csv',index=False)

--------------------------------
{'atributosDesejados__excluirName': True, 'classificador__criterion': 'gini', 'classificador__max_depth': 7, 'classificador__max_features': 'auto', 'classificador__min_samples_leaf': 1, 'classificador__min_samples_split': 4}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
